## ECO 542 PS #1 - More Harold Zurcher
### Import Data

In [622]:
using CSV, DataFrames, Statistics, LinearAlgebra
using Optim
file = CSV.read("./gradIOpset1data.csv");
first(file,5)

,busID,period,replace,state
,Int64⍰,Int64⍰,Bool⍰,Int64⍰
1,1,1,false,6
2,1,2,true,8
3,1,3,false,1
4,1,4,false,3
5,1,5,false,4


### Set up data & parameters

In [623]:
using Random, Distributions
Random.seed!(123) 
gumb = Gumbel()

d = zeros(200,120) #decisions
x = zeros(200,120) #states 
for busData in groupby(file, :busID)
    d[busData[:busID][1],:] = busData[:replace]
    x[busData[:busID][1],:] = busData[:state]
end

X = [0:20;]; # state space for bus
δ = 0.95;    # discount factor
# construct transition matrices
Γnr = [(i == j || i == j+2) ? 0.25 : (i == j+1) ? 0.5 : 0.0 for i=1:21,j=1:21];
Γnr[21,20] = 0.75; Γnr[21,21] = 1.0; # fix bottom right entries

Γr = zeros(21,21);
Γr[1,:] = [0.25 for i=1:21]; Γr[2,:] = [0.5 for i=1:21]; Γr[3,:] = [0.25 for i=1:21]; # manually code Γr

### 1. Building Blocks 

In [1039]:
function cvf(V,p1,b,c)
    v=zeros(2,21,2)
    EV = getEV(V)
    v[1,:,:] = b .* X + δ*EV[1,:,:]
    v[2,:,:] = c .* ones(21) + δ*EV[2,:,:]
    return v
end

function getEV(V)
    EV = zeros(2,21,2)
    ## do d=2 first, since the expected continuation value is constant
    for t=1:2
        EV[2,:,t] = dot(Γr[:,1],V[:,t]) .* ones(21)
        EV[1,:,t] = [dot(Γnr[:,i],V[:,1]) for i=1:21]
    end
    ## return 2x21x2 array of continuation values
    return EV
end

function emax(v)
    return(log.(exp.(v[1,:,:]) + exp.(v[2,:,:])))
end

function pchoose(v)
    p=zeros(2,21,2)
    num = exp.(v[2,:,:]-v[1,:,:])
    denom = ones(21,2)+num
    p[2,:,:] = num./denom
    p[1,:,:] = ones(21,2) - p[2,:,:]
    return p
end

function inversePchoose(p)
    #init v
    v=zeros(2,21,2)
    # since p = 1/(1+exp(v2-v1)) -> v2-v1 = log(p/1-p) inverse logistic
    dv =  log.(ones(21,2) -p[2,:,:]) - log.(p[2,:,:])

    # replace the engine resets the state of the bus, so v = V(0)
    v[1,:,:] = dv
    v2 = dv[1,1]-dv[1,2] # v(replace type 1) - v(replace type 2)
    # type 2 is more expensive, add back its value
    v[:,:,2] .+= v2 
    return v
end

function iterateValues(V,p1,b,c)
    return emax(cvf(V,p1,b,c)) 
end

function iterateCCP(p,p1,b,c)
    return pchoose(cvf(emax(inversePchoose(p)),p1,b,c))
end

function solveBellmanEquation(p1,b,c)
    Vold = zeros(21,2)
    Vnew = ones(21,2) 
    tol = 1e-14
    while (norm(Vnew - Vold) >= tol)
        Vold = Vnew
        Vnew = iterateValues(Vold,p1,b,c)
    end
    return Vnew
end

function loglikBusOfType(d,x,t,v)
    llh = 0
    p = pchoose(v)
    nPeriods = length(d)
    for i in 1:nPeriods
        if Int(d[i]) == 1
            llh += log(p[2, (Int(x[i])+1),t])
        end
        if Int(d[i]) == 0
            llh += log(p[1, (Int(x[i])+1),t])
        end 
    end
    return llh
end

loglikBusOfType (generic function with 1 method)

### 2. MLE/NFXP/Direct


In [173]:
function outerLoop(theta0)
    
    # transform p1 in (0,1)
    t = vcat(tanh(theta0[1]),theta0[2:5])
    
    # inner loop
    v = cvf(solveBellmanEquation(t[1],t[2:3]',t[4:5]'), t[1],t[2:3]',t[4:5]')
    
    #outerLoop -- evaluate likelihood given params
    return tot_llh(t[1],v)
end

function tot_llh(p1,v)
    # compute total sample llh by aggregating individ. bus llh
    tot_llh = 0
    for i=1:size(d,2)
        llh1 = exp(loglikBusOfType(d[i,:],x[i,:],1,v))
        llh2 = exp(loglikBusOfType(d[i,:],x[i,:],2,v))
        tot_llh = tot_llh + log(p1*llh1 + (1-p1)*llh2)
    end
    return tot_llh
end    

tot_llh (generic function with 1 method)

In [626]:
theta0 = [0.5 -1 -1 -10 -13]
@time res = optimize(z -> -outerLoop(z),theta0)
thetaHat = res.minimizer
thetaHat[1] = tanh(thetaHat[1])
@show thetaHat
V1 = solveBellmanEquation(thetaHat[1],thetaHat[2:3]',thetaHat[4:5]')
WTP2 = (ones(21)*V1[1,1]-V1[:,2])/abs(thetaHat[4]);

 13.256694 seconds (67.45 M allocations: 10.274 GiB, 10.65% gc time)
thetaHat = [0.433275 -0.864375 -1.05235 -11.1911 -13.7966]


21-element Array{Float64,1}:
 1.1568003221927245e-6
 0.33621828736457493  
 0.6093922947425133   
 0.8202732133875599   
 0.9767258519785401   
 1.0905341222151184   
 1.1686801085862901   
 1.2142440843897506   
 1.2356194209803975   
 1.2441335619747813   
 1.2472571058788688   
 1.2483663161223173   
 1.2487557200844135   
 1.248891905174314    
 1.2489394744850082   
 1.248956084100223    
 1.2489618829769726   
 1.2489639074645489   
 1.2489646142431714   
 1.2489648609898434   
 1.2489649471327324   

### 3. MPEC

In [1058]:
using GLPK, JuMP

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1186


In [1072]:
model = Model(with_optimizer(GLPK.Optimizer))
tol = 1e-2
@variable(model, norm(iterateValues(V, p1, b, c) - V) <= tol)
@objective(model, Max, tot_llh(theta, V, q))
function make_q(N)
    for i=1:q
    @variable(model, 0<=q<=1)
end

# @variable(model, q <= 1)

make_q (generic function with 1 method)

### 4. MLE/NFXP/EM
  

In [628]:
function Mstep(qList,beta)
    b = beta[1:2]'
    c = beta[3:4]'
    v = cvf(solveBellmanEquation(p1,b,c),p1,b,c)
    llhList = [loglikBusOfType(d[i,:],x[i,:],t,v) for i =1:size(d,1), t=1:2]
    llh = dot(qList,llhList[:,1]) + dot(ones(length(qList))-qList, llhList[:,2])
    return llh
end    

function Estep(p1,v)
    llhList = [exp(loglikBusOfType(d[i,:],x[i,:],t,v)) for i=1:size(d,1), t=1:2]
    qList = p1*llhList[:,1]./(p1 * llhList[:,1] + (1-p1)*llhList[:,2])
    return qList, mean(qList)
end

function EMloop()
    p1 = theta0[1]
    beta0=theta0[2:5]
    v0 = cvf(solveBellmanEquation(p1, theta0[2:3]',theta0[4:5]'),p1, theta0[2:3]',theta0[4:5]')
    vNew = similar(v0)
    tol = 1e-2
    its = 0 
    while (norm(v0-vNew) > tol)
        v0 = vNew
        its+=1
        qList, p1 = Estep(p1, vNew)
        res = optimize(beta -> -Mstep(qList, beta),  beta0, BFGS())
        beta0 = res.minimizer
        vNew = cvf(solveBellmanEquation(p1, beta0[1:2]',beta0[3:4]'),p1, beta0[1:2]',beta0[3:4]')
    end
    return p1, beta0, vNew
end

EMloop (generic function with 1 method)

In [629]:
@time p1_3, beta_3, v_3 = EMloop()
V3 = emax(vNew)
WTP3 = (ones(21)*V3[1,1]-V3[:,2])/abs(beta_3[3]);

 26.594832 seconds (127.52 M allocations: 20.170 GiB, 10.43% gc time)


### 5. Two-Step EM

In [699]:
function twoStepE(P, p1, beta)
    qList, p1 = Estep(p1,inversePchoose(P))
    Pnew = iterateCCP(P, p1,beta[1:2]',beta[3:4]')
    return qList,p1,Pnew
end

function twoStepM(qList, P, p1, beta)
    b = beta[1:2]'; c = beta[3:4]'
    v = inversePchoose(iterateCCP(P,p1,b,c))
    llhList = [loglikBusOfType(d[i,:],x[i,:],t,v) for i =1:size(d,1), t=1:2]
    llh = dot(qList,llhList[:,1]) + dot(ones(length(qList))-qList, llhList[:,2])
    return llh
end    

function twoStepEMloop(theta0)
    p1 = theta0[1]
    beta0 = theta0[2:5]
    P = pchoose(cvf(solveBellmanEquation(p1, theta0[2:3]',theta0[4:5]'),p1, theta0[2:3]',theta0[4:5]'))
    thetaNew = similar(theta0)
    tol = 1e-7
    while (norm(theta0-thetaNew) > tol)
        theta0 = thetaNew
        qList, p1, P = twoStepE(P, p1, beta0)
        res = optimize(b -> -twoStepM(qList, P, p1, b),  beta0, BFGS())
        beta0 = res.minimizer
        thetaNew= vcat(p1, beta0)
    end
    return p1, beta0, P
end

twoStepEMloop (generic function with 2 methods)

In [1047]:
theta0 = vcat(0.5, [-1, -1, -10, -13])
@time p1_5, beta_5, P_5 = twoStepEMloop(theta0)
V5 = emax(inversePchoose(P));
WTP5 = [(V5[1,1]-V5[x,2])/abs(beta_5[3]) for x=1:21];

  8.010754 seconds (15.94 M allocations: 6.362 GiB, 14.99% gc time)


## 6. Bayesian DDC 

In [1028]:
function weightV(θpropose, θcur, Vlist, M)
    if (size(Vlist,1) < M)
        return mean(Vlist)
    end
    θs = reverse(θpropose)
    θs = θs[1:M]
    weights = [norm(θs[i] - θcur) for i = 1:M]
    weights = weights./sum(weights)
    weightedV = sum([weights[i]*Vlist[i] for i = 1:M])
    return weightedV
end


function improveV(θpropose, theta0, Vlist, M)
    p1=theta0[1]
    b=theta0[2:3]'
    c=theta0[4:5]'
    Vtemp = getEV(weightV(θpropose,theta0, Vlist,M)) # equal weights on past V, NEED TO CHANGE
    epsilon = rand(gumb,2,21,2)
    u1 = b .* X
    u2 = c .* ones(21,2)
    Vnew = max.(u1 + epsilon[1,:,:] + δ*Vtemp[1,:,:], u2 + epsilon[2,:,:] + δ*Vtemp[2,:,:])
    pushfirst!(Vlist,Vnew) # add new to front
    while (length(Vlist) > M) # if list is full, 
        pop!(Vlist)  # drop oldest (last in list)
    end
    return Vnew, Vlist
end

function llh(θ2, V)
    p1=θ2[1]
    b=θ2[2:3]'
    c=θ2[4:5]'
    v=cvf(V, p1, b, c)
    return tot_llh(p1, v)
end

function π(theta)
    return sum([logpdf(Normal(theta[i+1],0.01), beta_5[i]) for i = 1:4])
    return 0 
end

function propose(θ2)
    # proposes θ* ~ g(θ1|θ2) 
    k=100
    betaNew = [rand(Normal(θ2[i+1],1/k)) for i=1:4]
    betallh = [logpdf(Normal(θ2[i+1],1/k), betaNew[i]) for i = 1:4]
     
    # ignore p1 for now (set equal to MLE from previous parts)
    p1New = p1_5 
    
    ## code for uniform draws
    #p1New = rand(Beta(1,1))
    # unif draws -> pdf = 1 at any pt, so log pdf = 0
    # p1llh = logpdf(Beta(1,1),p1New)
    
    θnew = vcat(p1New, betaNew)
    
    # g (thetaOld | thetaNew), if symmetric will be same as bettallh
    oldProp = [logpdf(Normal(betaNew[i],1/k), θ2[i+1]) for i = 1:4]
    
    # returns new draw, g(newDraw), g(oldDraw)
    return θnew, sum(betallh), sum(oldProp)
end

function metrop(θinit, M, nDraws)
   
    Vlist = []
    θlist = []
    θpropose = []
    accept = zeros(nDraws)
    
    burnIn = 100
    
    θold = θinit
    push!(Vlist,zeros(21,2))
    
    for i=1:nDraws
        # propose new θ'~ g(θ' | θ)
        θnew, propNew, propOld = propose(θold)
        push!(θpropose, θnew)
        
        #improve estimate of V
        Vnew, Vlist = improveV(θpropose, θnew, Vlist, M)
        
        # calculate jump pr in logs to avoid over/underflow
        
        # propNew = propOld because of symmetric density so we ignore
 
        num = π(θnew) + llh(θnew,V)
        denom = π(θold) + llh(θold,V)
        pJump = min(1, exp(num - denom))
   
        if isnan(pJump) 
            pJump = 0 
        end
        
        if rand(Bernoulli(pJump)) == 1
            push!(θlist,θnew)
            accept[i] = 1
            θold = θnew
        end
        # else θold not updated
    end
    
    return Vlist, θlist, θpropose, accept
end

metrop (generic function with 1 method)

In [1029]:
@time VlistNew, θlistNew, θproposeNew, acceptNew = metrop(vcat(p1_5, beta_5), 50, 10000);
thetaBayes = mean(θlistNew[100:end])
VBayes = solveBellmanEquation(thetaBayes[1],thetaBayes[2:3]',thetaBayes[4:5]')
WTPBayes = [(VBayes[1,1]-VBayes[x,2])/abs(thetaBayes[4]) for x=1:21];

 40.775579 seconds (111.90 M allocations: 35.869 GiB, 15.46% gc time)


In [1046]:
hcat(collect(0:20), WTP1,WTP3,WTP5, WTPBayes)

21×5 Array{Float64,2}:
  0.0  1.1568e-6  3.97916e-6  2.05678e-6  2.05678e-6
  1.0  0.336218   0.333461    0.314167    0.314167  
  2.0  0.609392   0.59057     0.566272    0.566272  
  3.0  0.820273   0.777625    0.757726    0.757726  
  4.0  0.976726   0.912399    0.898149    0.898149  
  5.0  1.09053    1.0143      1.00349     1.00349   
  6.0  1.16868    1.09233     1.08774     1.08774   
  7.0  1.21424    1.14601     1.15777     1.15777   
  8.0  1.23562    1.17622     1.21438     1.21438   
  9.0  1.24413    1.19018     1.256       1.256     
 10.0  1.24726    1.19587     1.28271     1.28271   
 11.0  1.24837    1.19805     1.29773     1.29773   
 12.0  1.24876    1.19887     1.30538     1.30538   
 13.0  1.24889    1.19917     1.30905     1.30905   
 14.0  1.24894    1.19928     1.31076     1.31076   
 15.0  1.24896    1.19932     1.31155     1.31155   
 16.0  1.24896    1.19934     1.3119      1.3119    
 17.0  1.24896    1.19934     1.31207     1.31207   
 18.0  1.24896    1.199